In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
# Load STRIPS data
strips_data = pd.read_csv("/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data.csv")

In [3]:
# Define BGM model class
class BGMModel:
    def __init__(self, vol, correlation, dt):
        self.vol = vol
        self.correlation = correlation
        self.dt = dt

    def simulate_forward_rates(self, initial_rates, num_steps, num_paths):
        num_rates = len(initial_rates)
        rates = np.zeros((num_paths, num_steps, num_rates))
        rates[:, 0, :] = initial_rates

        for t in range(1, num_steps):
            dW = np.random.multivariate_normal(mean=np.zeros(num_rates), cov=self.correlation * self.dt, size=num_paths)
            rates[:, t, :] = rates[:, t-1, :] + self.vol * rates[:, t-1, :] * dW

        return rates

In [ ]:

# Calibrate BGM model to market data
def calibrate_bgm(market_data):
    def error_function(params):
        vol, correlation = params[0], params[1]
        model = BGMModel(vol, correlation, dt=1/252)
        simulated_rates = model.simulate_forward_rates(initial_rates=market_data['Yield'].values, num_steps=10, num_paths=1000)
        model_prices = np.mean(np.exp(-np.cumsum(simulated_rates, axis=1)), axis=0)
        return np.sum((market_data['Price'].values - model_prices) ** 2)

    result = minimize(error_function, x0=[0.01, 0.5], bounds=[(0.001, 0.1), (0, 1)])
    return BGMModel(result.x[0], result.x[1], dt=1/252)

# Apply calibration and simulation
bgm_model = calibrate_bgm(strips_data)
simulated_rates = bgm_model.simulate_forward_rates(initial_rates=strips_data['Yield'].values, num_steps=10, num_paths=1000)

# Compute bond prices using discount factors
model_prices = np.mean(np.exp(-np.cumsum(simulated_rates, axis=1)), axis=0)

# Compare model prices with market prices
comparison_df = pd.DataFrame({
    'Maturity': strips_data['Maturity'],
    'Market Price': strips_data['Price'],
    'Model Price': model_prices
})

# Calculate pricing errors
comparison_df['Pricing Error'] = comparison_df['Market Price'] - comparison_df['Model Price']

# Display results
print(comparison_df.head())